In [324]:
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askdirectory
import pandas as pd
import numpy as np
from datetime import datetime
import hashlib
from checksumdir import dirhash

In [117]:
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

<Details>
    <summary> DB Schemas </summary>

    
    # File Schema:
    *Key = checksum value of file
    *Path = Expected file path
    *Date = Date file added
    *Prev = Previous file key
    *Next = Next file key
    *Desc = Description of use of file
    *Change = Changes from previous version
    
    #Tags Schema:
    *Name = Tag name
    *Parents = List of parent tags
    *Child = list of child tags
    
    #Package Schema:
    *Package = Name of package
    *Path = File path for file
    *Name = Name of file
    *Latest = Checksum value of latest version
    
</Details>

In [93]:
file_columns = ["Key", "Path", "Date", "Desc", "Next", "Prev", 'Change']
tag_columns = ["Name", "Parent", "Child", "List of Files"]
package_columns = ['Package', 'Path', 'Name', 'Latest']

In [377]:
fileName = "test.h5"
file_df = pd.DataFrame(columns = file_columns)
tag_df = pd.DataFrame(columns = tag_columns)
package_df = pd.DataFrame(columns = package_columns)

In [149]:
save_path = askdirectory()
save_path = save_path + "/" + fileName

In [306]:
file_df, tag_df, package_df = loadData()

Load data


In [383]:
newFile('test', askopenfilename())

File already exists


In [381]:
#Add file to database
def newFile(desc, path):
    key = md5(path)
    if (not checkFile(path)):
        #Add new file
        file_df.loc[len(file_df)] = [key, path, datetime.today().strftime('%Y-%m-%d'), desc, None, 'N/A', 'File Created']
        if (desc in file_df["desc"].values):
            #Prompt to update if file is repeated
            print("Similar file found, replace?")
            #TODO add new value with prev value and update old entry's next value
        saveData(save_path)
    else:
        print("File already exists")

In [387]:
updateFile(askopenfilename(), md5("C:/Users/derew/PackageManager/button_clicked.png"), 'updated legal text')

In [389]:
file_df

,Key,Path,Date,Desc,Next,Prev,Change
0,8c15bb0116ebc79eafe8c664db282efa,C:/Users/derew/PackageManager/button_clicked.png,2024-06-20,test,f031ea32333d8c8477ff11b956de86e1,N/A,change
1,f031ea32333d8c8477ff11b956de86e1,C:/Users/derew/PackageManager/Data/test,2024-06-20,test,None,8c15bb0116ebc79eafe8c664db282efa,updated legal text


In [369]:
#Update existing file in db
def updateFile(path, prevkey, changes):
    key = md5(path)
    if (not checkFile):
        print("file already exists")
    else:
        #Add new file linking previous entry
        prevdesc = file_df[file_df['Key'] == prevkey]["Desc"][0]
        file_df.loc[len(file_df)] = [key, path, datetime.today().strftime('%Y-%m-%d'), prevdesc, None, prevkey, changes]
        #update old value
        old_row = file_df.loc[file_df["Key"] == prevkey].index[0]
        file_df.loc[old_row, "Next"] = key

In [12]:
#Get hash value of file
def md5(file_name):
    m = hashlib.md5()
    with open(file_name, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            m.update(chunk)
        return m.hexdigest()

In [300]:
def compareFolder(path1, path2):
    return dirhash(path1, 'md5') == dirhash(path2, 'md5')

In [304]:
compareFolder('C:/Users/derew/PackageManager/Data', 'C:/Users/derew/PackageManager/Data2')

True

In [314]:
#Check to see if file already exists in database or not    
def checkFile(path):
    key = md5(path)
    return (key in file_df["Key"].values)

In [276]:
#Create folders
file_df

,Key,Path,Date,Desc,Next,Prev,Change
0,8c15bb0116ebc79eafe8c664db282efa,C:/Users/derew/PackageManager/button_clicked.png,2024-06-20,test,None,N/A,change


In [20]:
#Add file to DB
def addFile():
    print("select file to add")
    path = askopenfilename()
    file_name = path.split("/")[-1]
    key = (md5(path))
    #TODO Check existing file
    #If yes -> file preview, ELSE
    #TODO Ask for tags
    #Add to existing project -> File name (new or updated)
    #Get date
    #If update -> get old id, set old next to key
    #Save changes in db
    saveData()

In [22]:
def resetData():
    file_df = pd.DataFrame(Data = None, index = file_columns)
    tag_df = pd.DataFrame(Data = None, index = tag_columns)
    package_df = pd.DataFrame(Data = None, index = package_columns)
    fileName = ""

In [123]:
#Load file from existing file
def loadData():
    #TODO Verify with test data
    print("Load data")
    fileName = path.split("/")[-1]
    file_df = pd.read_hdf(path, key="files", append = True)
    tag_df = pd.read_hdf(path, key="tags", append = True)
    package_df = pd.read_hdf(path, key='packages', append = True)
    
    return file_df, tag_df, package_df

In [154]:
#Save data to file
def saveData(save_path):
    print("Saving data")
    file_df.to_hdf(save_path, key = 'files', mode='a', complib='zlib', complevel=5)
    tag_df.to_hdf(save_path, key = 'tags', mode='a', complib='zlib', complevel=5)
    package_df.to_hdf(save_path, key = 'packages', mode='a', complib='zlib', complevel=5)
    print("Data Saved")